## Using CNN and NCM for CIFAR-10 feature extraction and classification

### Summary
CIFAR-10 데이터셋을 분류하기 위해 CNN을 이용하여 데이터셋에 대한 특징을 추출 후 NCM을 이용하여 각 class에 대한 평균 값을 이용하여 분류

<span style="color: #2D3748; background-color:#fff5b1;">Test size를 0.2로 10번 반복 실험한 결과 평균적으로 0.34의 정확도를 보여주고 있고, test 데이터 1개를 분류하는데 0.0011초의 시간이 걸린다.</span>

In [1]:
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from utils.util import Info
from sklearn.neighbors import NearestCentroid
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from utils.NCM_Classifier import train, validate
from models.resnet_feature import resnet18_feature

c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
class Config(Info):
    def __init__(self):
        super(Info, self).__init__()
        self.device = 'PC'
        self.dataset = 'CIFAR_10'
        self.test_size = 0.2
        self.feature_size = 3072
        self.method = 'NCM'
        self.distance = 'Euclidean'
        self.reduction_method = [None, None] # method, n_components
        self.iter = 10

In [3]:
cig = Config()
cig.info()
cig.print_rutin()

Device ── PC
│
├──Dataset
│    └────CIFAR_10
│    └────Train size 80%
│    └────Feature size: 3072
│
├──Method
│    └────NCM
│    └────Euclidean
│
├──Dimension reduction
│    └────Method: None
│    └────Component size: None
│    └────Feature Reduction Ratio: None%
│
└──Iteration
    └────10
PC - CIFAR_10(80%) - NCM - 10 iteration


In [4]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

## Load CIFAR-10 Dataset

In [5]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                ])

batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

validationset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
val_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=2)

print(trainset.data.shape)
print(validationset.data.shape)

Files already downloaded and verified
Files already downloaded and verified
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

epoch = 100
num_class = 10

model = resnet18_feature(num_classes=num_class)
model.to(device)

classifier = NearestCentroid()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [7]:
best_acc = 0.

for idx in range(epoch):  # loop over the dataset multiple times
    # train for one epoch
    train(train_loader, model, classifier, criterion, optimizer, idx, device)

    # evaluate on validation set
    acc = validate(val_loader, model, classifier, criterion, device)

    # remember best acc@1 and save checkpoint
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)

    scheduler.step()

print('Finished Training')

Epoch: [0]
tensor([2., 9., 4., 9., 4., 2., 5., 8., 6., 9., 9., 2., 7., 8., 4., 3., 2., 9.,
        7., 4., 2., 2., 9., 2., 0., 5., 3., 0., 6., 9., 2., 6., 4., 4., 9., 8.,
        9., 2., 0., 6., 0., 4., 0., 2., 4., 8., 9., 9., 9., 4., 0., 7., 4., 1.,
        4., 1., 8., 4., 8., 9., 5., 2., 9., 8., 4., 4., 4., 8., 0., 2., 8., 2.,
        4., 4., 9., 1., 7., 9., 2., 6., 1., 8., 8., 2., 3., 4., 9., 1., 9., 6.,
        3., 4., 7., 2., 8., 6., 9., 4., 8., 2., 1., 1., 2., 1., 9., 0., 9., 0.,
        2., 3., 7., 9., 8., 9., 9., 9., 6., 8., 6., 8., 4., 3., 4., 4., 7., 2.,
        1., 8., 0., 6., 9., 4., 6., 4., 5., 8., 7., 9., 4., 4., 9., 2., 9., 1.,
        8., 4., 3., 0., 3., 2., 8., 8., 8., 0., 6., 4., 0., 2., 2., 7., 2., 0.,
        9., 7., 9., 9., 8., 7., 4., 6., 9., 5., 8., 9., 8., 8., 3., 9., 4., 8.,
        8., 8., 1., 8., 9., 9., 8., 3., 7., 3., 9., 2., 7., 8., 9., 0., 4., 4.,
        4., 4., 4., 7., 3., 8., 1., 4., 8., 9., 4., 7., 7., 7., 7., 8., 7., 7.,
        8., 6., 1., 0., 9., 4

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                      ])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                     ])

batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

validationset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
val_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=2)

print(trainset.data.shape)
print(validationset.data.shape)

Files already downloaded and verified
Files already downloaded and verified
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

epoch = 100
num_class = 10

model = resnet18_feature(num_classes=num_class)
model.to(device)

classifier = NearestCentroid()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [ ]:
best_acc = 0.

for idx in range(epoch):  # loop over the dataset multiple times
    # train for one epoch
    train(train_loader, model, classifier, criterion, optimizer, idx, device)

    # evaluate on validation set
    acc = validate(val_loader, model, classifier, criterion, device)

    # remember best acc and save checkpoint
    is_best = acc > best_acc
    best_acc = max(acc, best_acc)

    scheduler.step()

print('Finished Training')

TypeError: train() missing 1 required positional argument: 'device'